Instructions:
1. Runtime -> Run all
2. Cell before the last -> upload excel file
3. Introduce the file name, process variable to analyze, what is the time indicator and the LSL & USL
3. Report exported

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

! pip install fitter
from fitter import Fitter

!pip install pandas openpyxl
from google.colab import files


In [ ]:
# To read the excel file into a dataframe
def excel(dataset):
  df = pd.read_excel(dataset, engine='openpyxl')
  df.head(1)
  return df

In [ ]:
#Clean my table to have only size and PV data for 91232.2 and 902063
def clean_table(data,column_1,column_2,column_3):

  table = data[[column_1,column_2,column_3]]

  return table

In [ ]:
# Clean the data to be possible to apply the nelson rules
def clean_column(data, column_name):
    data_new = data.copy()
    # Convert the Series to DataFrame to use 'subset'
    data_new = pd.DataFrame(data_new)
    data_new.dropna(subset=[column_name], inplace=True)
    data_new= data_new[data_new[column_name] !=' ']
    return data_new

In [ ]:
# Function to evaluate if the particles are in spec for a certain property
def spec(data, column_x, LL, UL):

  data['Spec'] = 0
  # Iterate through each row of the DataFrame using the index
  for i in data.index:
    if (data.loc[i, column_x] >= LL) & (data.loc[i, column_x] <= UL):
      data.loc[i, 'Spec'] = 1 #pass
    else:  # Simplified the elif condition
      data.loc[i, 'Spec'] = 0 #fail

  return data

In [ ]:
def normality(df,column_name):
# Verify if you have a normal distribution
    data=df[column_name]
    data = pd.DataFrame(data)
    data.dropna(subset=[column_name], inplace=True)
    data= data[data[column_name] !=' ']
    data[column_name] = pd.to_numeric(data[column_name], errors='coerce')
    data = data[column_name].dropna().tolist() # Convert to list after ensuring numeric type

    data=list(data) # to use the next function it needs to be a list - need to conver from series to list

    # Visual check for normality
    stats.probplot(data, dist="norm", plot=plt)
    plt.title('Q-Q Plot')
    plt.show()
    # Numeric test for normality using D'Agostino and Pearson’s Test
    #s is the test statistic and p the p-value. We stablish we want a p value above 0.05

    s, p = stats.normaltest(data)

    if p > 0.05:
        print("Data follows a normal distribution (fail to reject H0) with p = %.3f" % (p))
    else:
        print("Data does not follow a normal distribution (reject H0)with p = %.3f" % (p))
    return data

In [ ]:
def nelson_rules(dat, column_name):
  # Create a copy of the database and clean it - remove all N/A
  data = dat.copy()
  # Convert the column to numeric, forcing errors to be NaN
  data[column_name] = pd.to_numeric(data[column_name], errors='coerce')
  data= clean_column(data,column_name)

  # Drop rows with any remaining non-numeric values in the target column
  data = data[pd.to_numeric(data[column_name], errors='coerce').notnull()]

  # Check for outliers
  Q1 = np.percentile(data[column_name], 25) # Calculate percentile on the numerical column
  Q3 = np.percentile(data[column_name], 75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  sample = data[(data[column_name] > lower_bound) & (data[column_name] < upper_bound)][column_name]

  center_line = (Q1+ 2* np.median(sample)+Q3)/4
  sigma = np.std(sample, ddof=0)

  # Nelson Rules:

    # Rule 1: One point outside 3-sigma control limits
  def rule1(series):
    test1 = (series > center_line + 3 * sigma) | (series < center_line - 3 * sigma)
    return test1

    # Rule 2: Nine points in a row on the same side of the center line
  def rule2(series):
    test2 = series.rolling(window=9).apply(
        lambda x: (x > center_line).all() or (x < center_line).all(), raw=True
        ).fillna(0).astype(bool)
    return test2

    # Rule 3: Six points in a row steadily increasing or decreasing
  def rule3 (series):
    test3 = series.rolling(window=6).apply(
        lambda x: all(x[i] < x[i + 1] for i in range(len(x) - 1)) or all(x[i] > x[i + 1] for i in range(len(x) - 1)),raw=True
        ).fillna(0).astype(bool)
    return test3

    # Rule 4: Fourteen points in a row alternating up and down
  def rule4(series):
    test4=series.rolling(window=14).apply(
        lambda x:all((x[i] > x[i-1] if i % 2 == 0 else x[i] < x[i-1]) for i in range(1, len(x))),
        raw=True).fillna(0).astype(bool)
    return test4

    #Rule 5: Two out of three points in a row in Zone A or beyond (2*sigma)
  def rule5(series):
    test5=series.rolling(window=3).apply(
        lambda x: (x > center_line + 2 * sigma).sum() + (x < center_line - 2 * sigma).sum() >= 2,
        raw=False
        ).fillna(0).astype(bool)
    return test5

    # Rule 6: Two out of three points in a row in Zone B or beyond (1* sigma)
  def rule6(series):
    test6=series.rolling(window=3).apply(
        lambda x: (x > center_line + sigma).sum() + (x < center_line - sigma).sum() >= 2,
        raw=False
        ).fillna(0).astype(bool)
    return test6

    # Rule 7: Fifteen points in a row within Zone C (both above and below center line) (under 1 sigma)
  def rule7(series):
    test7=series.rolling(window=15).apply(
        lambda x: all((center_line - sigma < val < center_line + sigma) for val in x),
        raw=False
        ).fillna(0).astype(bool)
    return test7

    # Rule 8: Eight points in a row with none in Zone C (under 1 sigma)
  def rule8(series):
    test8=series.rolling(window=8).apply(
        lambda x: all((x < center_line - sigma)| (x > center_line + sigma)),
        raw=False
        ).fillna(0).astype(bool)
    return test8

  # Apply rules
  data['R1'] = rule1(data[column_name])
  data['R2'] = rule2(data[column_name])
  data['R3'] = rule3(data[column_name])
  data['R4'] = rule4(data[column_name])
  data['R5'] = rule5(data[column_name])
  data['R6'] = rule6(data[column_name])
  data['R7'] = rule7(data[column_name])
  data['R8'] = rule8(data[column_name])

  return data

In [ ]:
# Graph the results in a process plot
def process_plot(df,x_axis,y_axis, LL, UL):

  plt.figure(figsize=(40, 6))  # figure size
  sns.scatterplot(x=x_axis, y=y_axis, data=df)
  sns.lineplot(x=x_axis, y=y_axis, data=df)
  plt.title(f'{x_axis} by {y_axis}')
  plt.xlabel(x_axis)
  plt.ylabel(y_axis)
  plt.xticks(rotation=45, ha='right')
  plt.axhline(y=LL, color='orange', linestyle='--', label='Lower Limit')
  plt.axhline(y=UL, color='orange', linestyle='--', label='Upper Limit')


  return plt.show()

In [ ]:
def nelson_plot(df,x_axis,y_axis):

  df = df[pd.to_numeric(df[y_axis], errors='coerce').notnull()]

  # Check for outliers
  Q1 = np.percentile(df[y_axis], 25) # Calculate percentile on the numerical column
  Q3 = np.percentile(df[y_axis], 75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  sample = df[(df[y_axis] > lower_bound) & (df[y_axis] < upper_bound)][y_axis]

  center_line = sample.median()
  sigma = np.std(sample, ddof=0)
  LL = center_line - 3 * sigma
  UL = center_line + 3 * sigma

  # Create the line plot
  plt.figure(figsize=(40, 6))
  sns.lineplot(x=x_axis, y=y_axis, data=df, marker='o', color='blue', label='Line Plot')

  # Create the scatter plot
  sns.scatterplot(x=x_axis, y=y_axis, data=df, color='blue', label='Scatter Plot')

  # Add points for Nelson Rules violations

  for index, row in df.iterrows():
      if any(row[['R1', 'R2', 'R3', 'R4', 'R5', 'R6', 'R7', 'R8']]): # Check if any rule is violated
          plt.scatter(row[x_axis], row[y_axis], color='red', marker='o', s=100)

  plt.axhline(y=LL, color='grey', linestyle='--', label='Lower Limit')
  plt.axhline(y=UL, color='grey', linestyle='--', label='Upper Limit')
  plt.axhline(y=center_line, color='grey', linestyle='--')

  plt.title(f'{x_axis} by {y_axis}')
  plt.xlabel(x_axis)
  plt.ylabel(y_axis)

  plt.xticks(rotation=45, ha='right')
  plt.legend()
  plt.show()
  return df

In [ ]:
# Multivariable analysis
def matrix_plot(data):

  def numeric_table(data):

    table = data.apply(pd.to_numeric, errors='coerce')

    return table

  # Create a multivariable matrix
  data = numeric_table(data)


  # Create a heatmap
  correlation_matrix = data.corr()

  plt.figure(figsize=(10, 8))
  sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
  plt.title('Correlation Matrix')

  return plt.show()



In [ ]:
# Process Capability
def process_capability(data1,column_name,LSL,USL):
  data = pd.to_numeric(data1[column_name], errors='coerce')
  data = data[np.isfinite(data)]

    # Define USL & LSL
  mean = np.mean(data)
  std_dev = np.std(data)

  Cp = (USL-LSL)/(6*std_dev)
  Cpl=(mean - LSL)/(3*std_dev)
  Cpu=(USL - mean)/(3*std_dev)
  Cpk=min(Cpl,Cpu)
  print(f"\033[1mCp: {Cp:.2f}\033[0m")
  print(f"\033[1mCpk: {Cpk:.2f}\033[0m")


  # Fit multiple distributions
  f = Fitter(data, distributions=['skewnorm', 'norm', 'expon', 'lognorm', 'chi2', 't'])
  f.fit()

  # Get the best distribution
  best_distribution = f.get_best(method='sumsquare_error')
  best_dist_name = list(best_distribution.keys())[0]
  best_params = best_distribution[best_dist_name]


  # Create the histogram
  plt.hist(data, bins=30, density=True, alpha=0.6, color='lightblue', edgecolor='black', linewidth=1.2)

  # Generate the best fit line
  x = np.linspace(min(data), max(data), 1000)
  if best_dist_name == 'skewnorm':
      plt.plot(x, stats.skewnorm.pdf(x, *best_params.values()), 'black', linewidth=2)
  elif best_dist_name == 'norm':
      plt.plot(x, stats.norm.pdf(x, *best_params.values()), 'black', linewidth=2)
  elif best_dist_name == 'expon':
      plt.plot(x, stats.expon.pdf(x, *best_params.values()), 'black', linewidth=2)
  elif best_dist_name == 'lognorm':
      plt.plot(x, stats.lognorm.pdf(x, *best_params.values()), 'black', linewidth=2)
  elif best_dist_name == 'chi2':
      plt.plot(x, stats.chi2.pdf(x, *best_params.values()), 'black', linewidth=2)
  elif best_dist_name == 't':
      plt.plot(x, stats.t.pdf(x, *best_params.values()), 'black', linewidth=2)

  # Add title and labels
  plt.title(f'Histogram with Best Fit: {best_dist_name}')
  plt.xlabel('Data values')
  plt.ylabel('Density')
  plt.show()

  print(f"Best distribution: {best_dist_name}")
  print(f"Parameters: {best_params}")

  return data


In [ ]:
# Import your spreadsheet into excel - change name here:

def report(data, column_name2, column_name1, LSL, USL):
    # Process the data
    print(f"\033[1mProcess analysis report for variable {column_name2}\033[0m")


    A = excel(data)

    print(f"""
    - Normality:
    If the distribution is not normal, the methods here applied may not be valid.
    """)

    B = normality(A, column_name2)

    C = nelson_rules(A, column_name2)

    print(f"""
    - Process Capability:
    Ideally Cpk > 1.33
    """)

    E = spec(C, column_name2, LSL, USL)
    csv_path = 'report.csv'
    E.to_csv(csv_path, index=False)
    files.download(csv_path)

    F = process_plot(C, column_name1, column_name2, LSL, USL)

    I = process_capability(A, column_name2, LSL, USL)



    print(f"""
    - Nelson rules test:
    """)


    D = nelson_plot(C, column_name1, column_name2)

    print(f"""
    - Multivariant analysis:
    """)

    G = C.iloc[:, :10]
    H = matrix_plot(G)


    return B, E, F, I, H

In [ ]:
def user():
  data = input("Enter the name of the excel file: ")
  column_name2 = input("Enter the name of the column to analyze: ")
  column_name1 = input("Enter the name of the column for grouping: ")
  LSL = float(input("Enter the lower limit: "))
  USL = float(input("Enter the upper limit: "))
  report(data,column_name2,column_name1, LSL,USL)

  print(report)

if __name__ == "__user__":
  user()

In [ ]:
# Upload excel files to the colab
uploaded = files.upload()

In [ ]:
# Get the report
Report = user()